In [1]:
import pandas as pd

train_images_path = "/home/docker_current/datasets/train"
df_train = pd.read_csv("/home/docker_current/datasets/train.csv")
df_train['text_features'] = None
df_train['image_features'] = None
df_train['pred_id'] = 0
df_train.head(3)

,id,description,object_img,text_features,image_features,pred_id
0,520,Фотография. Г. Пермь. Здание горисполкома. ПО...,799,None,None,0
1,1817,Фотонегатив пленочный. Труппа театра «У моста»...,854,None,None,0
2,188,"Изразец гладкий расписной ""пермский""- карниз А...",1794,None,None,0


In [2]:
'''Загрузка multilangual-CLIP предиктора'''
# from MLCLIP import MLCLIPPredictor
# predictor = MLCLIPPredictor()

'Загрузка multilangual-CLIP предиктора'

In [3]:
'''Загрузка ruCLIP предиктора'''
# from ruCLIP import RUCLIPPredictor
# predictor = RUCLIPPredictor()

'Загрузка ruCLIP предиктора'

In [4]:
'''Загрузка TranslateCLIP предиктора'''
from TranslateCLIP import TranslateCLIP
predictor = TranslateCLIP()

In [18]:
from PIL import Image

ind = 5
text = df_train['description'][ind]
sample_image_path = "/home/docker_current/datasets/train/" + str(df_train['object_img'][ind]) + ".png"
image = Image.open(sample_image_path)

text_features, image_features = predictor.inference(text, image)
image_features.shape, text_features.shape

((1, 1024), (1, 1024))

In [19]:
import torch
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

cos(torch.Tensor(text_features), torch.Tensor(image_features))

tensor([0.3535])